In [ ]:
import pandas as pd

In [ ]:
columns = ['Database Name', 'Patient Number', 'Client Number', 'Consultation Number', 'Item Name', 'Units',
           'Item Label', 'Clinic Code', 'Consulting Vet', 'Patient Desexed', 'Deceased Date',
           'Visit Date', 'Consultation Date', 'Examination Text' ,'Patient Species', 'Patient Breed',
           'Patient Colour', 'Patient Sex', 'Patient Date of Birth', 'Insured', 'weight', 'temperature',
           'HeartRate', 'BodyScore', 'DentalGrade', 'RespRate', 'PainScore', 'BP', 'CRT', 'MMColour']

vitalsigns = ['weight', 'temperature', 'HeartRate', 'BodyScore', 'DentalGrade', 'RespRate', 'PainScore', 'BP', 'CRT', 'MMColour']

In [ ]:
data_dir = '../Data/vet_compass/'
data = pd.read_csv(data_dir + 'STR024A03 20190719 sample x10,000.csv', names = columns)

In [ ]:
for sign in vitalsigns:
    print(data[sign].dtype)